### Imports and parameters

In [1]:
import cv2 as cv
import copy as cp
import numpy as np
import numpy.random as rnd
import PIL.Image as img
import PIL.ImageDraw as imgdr
import PIL.ImageChops as chops
import scipy.spatial.distance as scidst
from skimage.measure import compare_mse

#source_image = img.open("./source_img.png").convert('RGB')
#source_image = cv.imread("./res/source_img.png")
source_image = cv.imread("./res/penguin.jpeg")
source_image_size = (512,512)
generation_size = 5
max_generation_cycles = 500
mutation_chance = 0.25
mutation_speed = 10.0
mutation_amount = 10
crossover_amount = 0.5 #percents
selection_percent = 0.8

#source_image_arr = np.array(source_image).reshape(512*512*4)
#source_image_norm = np.sum(source_image_arr)

### Image Utils

In [2]:
def distance(image1,image2):
    return compare_mse(image1,image2)#scidst.cosine(img1_arr,img2_arr)

def get_random_color():
    #if transparency == 0:
        #transparency = 255#np.random.randint(0,255)
    return (np.random.randint(256), np.random.randint(256),np.random.randint(256))#np.random.randint(256))#,transparency)

# def change_color(color,dcomp):
#     r,g,b = color
#     r += int(np.clip(0,rnd.uniform(-dcomp,dcomp),255))
#     g += int(np.clip(0,rnd.uniform(-dcomp,dcomp),255))
#     b += int(np.clip(0,rnd.uniform(-dcomp,dcomp),255))
#     #a += int(np.clip(0,rnd.uniform(-dcomp,dcomp),255))
#     return (r,g,b)

### Mutations

In [3]:
class CircleMutation:
    
    def __init__(self,x,y,radius,color):
        self.x, self.y = int(x), int(y)
        self.radius = int(radius)
        self.color = color
        
    def apply(self,genes):
        cv.circle(genes,(self.x,self.y),self.radius,self.color,thickness=cv.FILLED)
    
    @staticmethod
    def generate_random():
        w,h = source_image.shape[0],source_image.shape[1]
        x = np.random.randint(0,w)
        y = np.random.randint(0,h)
        #r = np.random.exponential(20,4)
        r = np.random.randint(0,6)
        gene = CircleMutation(x,y,r,get_random_color())
        return gene
    
class PolygonMutation:
    
    def __init__(self,points,color):
        self.points = points
        self.color = color
    
    def apply(self,genes):
        cv.fillPoly(genes,[self.points],self.color)
        
    @staticmethod
    def generate_random():
        vertices = 3
        w,h = source_image.shape[0],source_image.shape[1]
        rnd_point = [np.random.uniform(0,w),np.random.uniform(0,h)]
        size = [20,30]
        points = []
        for i in range(vertices):
            point_x = int(np.clip(0,rnd_point[0]+np.random.uniform(-1,1)*size[0],w))
            point_y = int(np.clip(0,rnd_point[1]+np.random.uniform(-1,1)*size[1],h))
            points.append([point_x,point_y])
        return PolygonMutation(np.array(points,dtype=np.int32),get_random_color())
    
class SquareMutation:
    
    def __init__(self,x,y,side,color):
        self.x, self.y = int(x), int(y)
        self.hside = int(side/2)
        self.color = color
        
    def apply(self,genes):
        start_point = (self.x-self.hside,self.y-self.hside)
        end_point = (self.x+self.hside,self.y+self.hside)
        cv.rectangle(genes,start_point,end_point,self.color,thickness=cv.FILLED)
    
    @staticmethod
    def generate_random():
        w,h = source_image.shape[0],source_image.shape[1]
        x = np.random.randint(0,w)
        y = np.random.randint(0,h)
        side = np.random.randint(0,6)
        gene = SquareMutation(x,y,side,get_random_color())
        return gene

### Chromosome

In [4]:
class Chromosome:
    
    def __init__(self,genes=None):
        if genes is None:
            genes = np.zeros_like(source_image)
        self.genes = genes
        self.fitness = self.get_fitness(source_image)
    
    def get_fitness(self,target_image):
        self.fitness = 1/(1 + distance(target_image,self.genes))
        return self.fitness
    
    def mutate(self):
        new_genes = self.genes.copy()
        for i in range(8):
            choice = np.random.randint(0,3)
            mutation = None
            if choice==0:
                mutation = CircleMutation.generate_random()
            if choice==1:
                mutation = SquareMutation.generate_random()
            if choice==2:
                mutation = PolygonMutation.generate_random()
            mutation.apply(new_genes)
        new_fitness = 1/(1 + distance(new_genes,source_image))
        if new_fitness > self.fitness:
            self.genes = new_genes
            self.fitness = new_fitness
    
    @staticmethod
    def cross(chr1,chr2):
        w_sum = chr1.fitness + chr2.fitness
        w1, w2 = (chr1.fitness,chr2.fitness) / w_sum
        #result = cv.add(chr1.genes,chr2.genes)
        #result = cv.addWeighted(chr1.genes,w1,chr2.genes,w2,0)
        result = cv.addWeighted(chr1.genes,0.5,chr2.genes,0.5,0)
        return Chromosome(result)

    def copy(self):
        return Chromosome(self.genes.copy())
    
    def __lt__(self, other):
        return self.fitness < other.fitness
            
    def __eq__(self, other):
        return self.fitness == other.fitness
        
    def __hash__(self):
        return int(1000000000*self.fitness)
    
    def __str__(self):
        return "Chromosome :fitness = {}".format(self.fitness)
    
p_mutate = np.vectorize(Chromosome.mutate)

### GA Engine

In [5]:
def do_mutate(generation):
    p_mutate(generation)
    #for i in range(len(generation)):
     #   generation[i].mutate()
      
    
def do_crossover(generation,crossover_percent=0.7):
    limit = int(crossover_percent*len(generation))
    result_generation = []
    for i in range(1,limit):
        result_generation.append(Chromosome.cross(generation[i-1],generation[i]))
    return result_generation


def do_select(generation,select_percent=0.8):
    limit = int(select_percent*len(generation))
    generation[::-1].sort()
    return generation[:limit]


def do_simulate(population_size = 40,population_limit=200000):
    generation = np.array([Chromosome() for i in range(population_size)])
    best_image = generation[0].genes
    best_fitness = generation[0].fitness
    for i in range(population_limit):
        new_generation = do_crossover(generation,0.30)
        generation = np.concatenate((generation,new_generation))
        do_mutate(generation)
        generation = do_select(generation,0.9)
        if generation[0].fitness > best_fitness:
            best_fitness = generation[0].fitness
            best_image = generation[0].genes
        if len(generation) > population_size:
            generation = generation[:population_size]
        print("Generation {} : best fitness {}".format(i,best_fitness))
        if i % 10 == 0:
            cv.imwrite("./out/result{}.png".format(i),best_image)

### Main

In [6]:
def main():
    try:
        chromosome = do_simulate(40)
        return chromosome.image
    except KeyboardInterrupt:
        print("interrupted")
    
main()

Generation 0 : best fitness 2.2663626027061973e-05
Generation 1 : best fitness 2.275660189107287e-05
Generation 2 : best fitness 2.282310665491976e-05
Generation 3 : best fitness 2.2961184621827428e-05
Generation 4 : best fitness 2.3045075941058963e-05
Generation 5 : best fitness 2.3137717189985462e-05
Generation 6 : best fitness 2.3273279925657865e-05
Generation 7 : best fitness 2.3367950578729004e-05
Generation 8 : best fitness 2.3478033595736623e-05
Generation 9 : best fitness 2.3691898719014997e-05
Generation 10 : best fitness 2.380684803165682e-05
Generation 11 : best fitness 2.3929224800980708e-05
Generation 12 : best fitness 2.39692723291737e-05
Generation 13 : best fitness 2.40835640684318e-05
Generation 14 : best fitness 2.4153355760400896e-05
Generation 15 : best fitness 2.4306195582402274e-05
Generation 16 : best fitness 2.4469073258198633e-05
Generation 17 : best fitness 2.4546192245534963e-05
Generation 18 : best fitness 2.4680148869277654e-05
Generation 19 : best fitness 

Generation 159 : best fitness 4.2238347106025696e-05
Generation 160 : best fitness 4.2314842213695196e-05
Generation 161 : best fitness 4.245403530149028e-05
Generation 162 : best fitness 4.2547355357538e-05
Generation 163 : best fitness 4.276491574277811e-05
Generation 164 : best fitness 4.2933151526972865e-05
Generation 165 : best fitness 4.299907779348066e-05
Generation 166 : best fitness 4.3164114302745134e-05
Generation 167 : best fitness 4.326674116307655e-05
Generation 168 : best fitness 4.3403465738559024e-05
Generation 169 : best fitness 4.3514479406669755e-05
Generation 170 : best fitness 4.36663889016178e-05
Generation 171 : best fitness 4.3783450268005556e-05
Generation 172 : best fitness 4.395041308793648e-05
Generation 173 : best fitness 4.403797312523356e-05
Generation 174 : best fitness 4.413817814000806e-05
Generation 175 : best fitness 4.419015214211844e-05
Generation 176 : best fitness 4.4369816406824714e-05
Generation 177 : best fitness 4.442964510349499e-05
Generat

Generation 316 : best fitness 6.2540485090553e-05
Generation 317 : best fitness 6.26597408249298e-05
Generation 318 : best fitness 6.281715148349562e-05
Generation 319 : best fitness 6.288277373884628e-05
Generation 320 : best fitness 6.298748948521556e-05
Generation 321 : best fitness 6.308944162563628e-05
Generation 322 : best fitness 6.319841111322946e-05
Generation 323 : best fitness 6.338811105616918e-05
Generation 324 : best fitness 6.347864666689746e-05
Generation 325 : best fitness 6.37163816111572e-05
Generation 326 : best fitness 6.379751490121823e-05
Generation 327 : best fitness 6.398814125504001e-05
Generation 328 : best fitness 6.403439319150445e-05
Generation 329 : best fitness 6.4104194983567e-05
Generation 330 : best fitness 6.429019945177975e-05
Generation 331 : best fitness 6.43686951640673e-05
Generation 332 : best fitness 6.45355679371575e-05
Generation 333 : best fitness 6.469458423754735e-05
Generation 334 : best fitness 6.473623417536944e-05
Generation 335 : bes

Generation 475 : best fitness 8.097299662225468e-05
Generation 476 : best fitness 8.109636707464988e-05
Generation 477 : best fitness 8.126057985792463e-05
Generation 478 : best fitness 8.132193864101146e-05
Generation 479 : best fitness 8.140804205488573e-05
Generation 480 : best fitness 8.152762954842832e-05
Generation 481 : best fitness 8.166083549873441e-05
Generation 482 : best fitness 8.179119119366815e-05
Generation 483 : best fitness 8.191499833846394e-05
Generation 484 : best fitness 8.191499833846394e-05
Generation 485 : best fitness 8.203186278386516e-05
Generation 486 : best fitness 8.219248626850596e-05
Generation 487 : best fitness 8.2269081682182e-05
Generation 488 : best fitness 8.231004812094121e-05
Generation 489 : best fitness 8.241939475533688e-05
Generation 490 : best fitness 8.260710036449454e-05
Generation 491 : best fitness 8.27005543564965e-05
Generation 492 : best fitness 8.278052344300758e-05
Generation 493 : best fitness 8.288057311680241e-05
Generation 494 

Generation 633 : best fitness 9.668064552906982e-05
Generation 634 : best fitness 9.681984729012911e-05
Generation 635 : best fitness 9.689599332332355e-05
Generation 636 : best fitness 9.706045811063102e-05
Generation 637 : best fitness 9.706184048031437e-05
Generation 638 : best fitness 9.709828777674313e-05
Generation 639 : best fitness 9.724476167655008e-05
Generation 640 : best fitness 9.733592475479725e-05
Generation 641 : best fitness 9.75525848386402e-05
Generation 642 : best fitness 9.767368320158467e-05
Generation 643 : best fitness 9.767368320158467e-05
Generation 644 : best fitness 9.76971941541796e-05
Generation 645 : best fitness 9.77700769255192e-05
Generation 646 : best fitness 9.783177369593499e-05
Generation 647 : best fitness 9.786372867569329e-05
Generation 648 : best fitness 9.8007066618168e-05
Generation 649 : best fitness 9.80192674078103e-05
Generation 650 : best fitness 9.807195042572346e-05
Generation 651 : best fitness 9.818272810609316e-05
Generation 652 : b

Generation 789 : best fitness 0.00010984105214166541
Generation 790 : best fitness 0.00010995316216503385
Generation 791 : best fitness 0.00011004598180517913
Generation 792 : best fitness 0.00011010594669859425
Generation 793 : best fitness 0.0001101490559165066
Generation 794 : best fitness 0.00011018542101991414
Generation 795 : best fitness 0.00011029996002121546
Generation 796 : best fitness 0.0001104232942454812
Generation 797 : best fitness 0.00011046251525796043
Generation 798 : best fitness 0.00011054139103875143
Generation 799 : best fitness 0.00011059765381600235
Generation 800 : best fitness 0.00011067820723486306
Generation 801 : best fitness 0.00011075094387446571
Generation 802 : best fitness 0.00011088737100895775
Generation 803 : best fitness 0.00011096905974987432
Generation 804 : best fitness 0.0001113794623776437
Generation 805 : best fitness 0.0001113794623776437
Generation 806 : best fitness 0.00011140967542822281
Generation 807 : best fitness 0.000111409675428222